In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


X = tf.placeholder(tf.float32, [None, 784])
y_true = tf.placeholder(tf.float32, [None, 10])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


__把加layer寫成一個function__

這編寫的add_layer其實只是幫我做了把input資料，乘上一個W矩陣加上bias，也就是一層fully connected層的意思，最後activation function再經過一個softmax，最後輸出就是網路的輸出，已經可以和ground truth label算cross entropy

__NOTE: __裡面的 __W__ 和 __bias__ 宣告的方式會影響其初始化的方式，往往對訓練品質會有很大的影響，需要作好的調整

In [61]:
def add_layer(inputs, in_size, out_size, activation_function=None):
    """
    args1 input: 輸入input的placeholder
    args2 in_size: input維度
    args3 out_size: output維度
    args4 activation_function: 激勵函數，通常是tf.nn下的函式
    """
    W = tf.Variable(tf.zeros([in_size, out_size]))
    bias = tf.Variable(tf.zeros([out_size]) + 0.1)
    WX_plus_b = tf.matmul(inputs, W) + bias
    if activation_function is None:
        outputs = WX_plus_b
    else:
        outputs = activation_function(WX_plus_b)
    return outputs

__ML第一步-定義模型__

這邊使用寫好的add_layer函式，使用softmax函式，輸入維度維784，輸出維度為10

In [62]:
hidden_layer_1_output = add_layer(X, 784, 512, activation_function=tf.nn.relu)
hidden_layer_2_output = add_layer(hidden_layer_1_output, 512, 512, activation_function=tf.nn.relu)
output = add_layer(hidden_layer_2_output, 512, 10, activation_function=tf.nn.softmax)

In [63]:
# output = add_layer(X, 784, 10, activation_function=tf.nn.softmax)

ML第二步-定義Loss Function

In [64]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_true * tf.log(output), reduction_indices=[1]))

ML第三步-Gradient Descent

In [65]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

--------

寫一個evaluation function方便之後計算accuracy

In [66]:
# return accuracy
def evaluation(feed_dict):
    correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y_true, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return sess.run(accuracy, feed_dict = feed_dict)

開始執行computational graph

In [67]:
init = tf.global_variables_initializer()

In [68]:
sess = tf.Session()

In [69]:
sess.run(init)

In [70]:
from time import time

start = time()

for i in range(2000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
#     sess.run(train_step, feed_dict={X: batch_xs, y_true: batch_ys})
    """
    如果執行sess.run的時候，也一起執行cross_entropy這個node
    則sess.run可以回傳「當次iteration的loss」，如以下程式碼
    """
    _, current_loss = sess.run([train_step, cross_entropy], feed_dict={X: batch_xs, y_true: batch_ys})

    if i % 50 == 0:
        print('第', i, '輪訓練的loss為', current_loss, ', accuracy為', evaluation({X: batch_xs, y_true: batch_ys}))
        
end = time()
print('訓練花費', (end-start), '秒')

第 0 輪訓練的loss為 2.30259 , accuracy為 0.14
第 50 輪訓練的loss為 2.30282 , accuracy為 0.09
第 100 輪訓練的loss為 2.30188 , accuracy為 0.17
第 150 輪訓練的loss為 2.30286 , accuracy為 0.1
第 200 輪訓練的loss為 2.30212 , accuracy為 0.12
第 250 輪訓練的loss為 2.30082 , accuracy為 0.19
第 300 輪訓練的loss為 2.30413 , accuracy為 0.07
第 350 輪訓練的loss為 2.30065 , accuracy為 0.15
第 400 輪訓練的loss為 2.30173 , accuracy為 0.14
第 450 輪訓練的loss為 2.30194 , accuracy為 0.12
第 500 輪訓練的loss為 2.30157 , accuracy為 0.13
第 550 輪訓練的loss為 2.2985 , accuracy為 0.18
第 600 輪訓練的loss為 2.29987 , accuracy為 0.14
第 650 輪訓練的loss為 2.2992 , accuracy為 0.16
第 700 輪訓練的loss為 2.30289 , accuracy為 0.12
第 750 輪訓練的loss為 2.2994 , accuracy為 0.14
第 800 輪訓練的loss為 2.30596 , accuracy為 0.05
第 850 輪訓練的loss為 2.30047 , accuracy為 0.1
第 900 輪訓練的loss為 2.30499 , accuracy為 0.08
第 950 輪訓練的loss為 2.30488 , accuracy為 0.1
第 1000 輪訓練的loss為 2.30314 , accuracy為 0.13
第 1050 輪訓練的loss為 2.29949 , accuracy為 0.12
第 1100 輪訓練的loss為 2.29903 , accuracy為 0.11
第 1150 輪訓練的loss為 2.30089 , accuracy為 0.09
第 1200 輪訓練的loss為 2.30

---

evaluation

In [71]:
correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y_true, 1)) # 注意這邊的 y 和 y_ 都是placeholder
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Testing Data Accuracy: ', evaluation({X: mnist.test.images, y_true: mnist.test.labels}))
print('Training Data Accuracy: ', evaluation({X: mnist.train.images, y_true: mnist.train.labels}))

Testing Data Accuracy:  0.1135
Training Data Accuracy:  0.112345


In [73]:
sess.close()